## 如何打开和读取文本文件的内容
- 使用open函数或者with open
- 然后调用以下几个方法
    - read():读取文件的全部内容，如果指定参数n，会读取前n个字符
    - readline():按行读取，如果指定参数n，如果当前行字符个数比n大，那么返回当前行的n个字符，如果比n小，返回所有字符
    - readlines():返回所有行组成的一个列表，如果指定参数n，该n表示行与行加起来的字符数，如果n大于第一行的数且小于第二行+第一行的字符数，那么返回结果是第一行和第二行组成的列表

## 使用open函数打开文件，并返回一个IO对象，，，该对象有3个读取文件的方法，read，readline和readlines。请用代码描述。三个方法的区别

In [23]:
with open("test.txt", "w", encoding="utf-8") as fp:
    fp.write("我喜欢你\n")
    fp.write("不过也许没那么喜欢\n")
    fp.write("我也不知道我是不是真的还有\n")
    fp.write("一颗心来喜欢一个人\n")
    fp.write("也许我的心早就死了")
    print("写入完成")
    
    
with open("test.txt", "r", encoding="utf-8") as fp:
# #     con = fp.read()
#     con = fp.read(10)  # \n换行符也算一个字符
#     print(con)

#     con = fp.readline()
#     print(con, end="")
#     con1 = fp.readline()
#     con2 = fp.readline(3)
#     con3 = fp.readline(20)  # 当n很大时，只返回改行的所有字符
#     print(con1, end="")
#     print(con2, end="")
#     print(con3, end="")
    
#     con =  fp.readlines()
#     print(con)
    con1 = fp.readlines(15)  #15大于了第一行和第二行字符数之和，所以返回到第三行
    print(con1)

写入完成
['我喜欢你\n', '不过也许没那么喜欢\n', '我也不知道我是不是真的还有\n']


## 在json序列化时，可以自动处理哪些数据类型
- str, int, tuple, list, dict, bool, None
- 但datetime不支持json序列化


## 在json序列化时，如何处理日期类型
- 需要在转换类的default方法中手工完成对日期类型值的处理

In [32]:
# json序列化处理日期类型，需要自定义类来解决
import json
from datetime import datetime, date


class DateToJson(json.JSONEncoder):
    # 有一个default函数
    def default(self, obj):
        if isinstance(obj, datetime):
            return obj.strftime("%Y-%m-%d %H:%M:%S")  # 如果使用年月日会报错
        elif isinstance(obj, date):
            return obj.strftime("%Y-%m-%d")
        else:
            return json.JSONEncoder.default(self, obj)


d = {'name': 'jioajioa', 'date': datetime.now()}
print(json.dumps(d, cls=DateToJson, ensure_ascii=False))

{"name": "jioajioa", "date": "2021-02-21 12:39:19"}


## with语句有什么用，请用代码解释
- with语句可以确保不管是否抛出异常，都会释放资源

## 如何将with语句用于一个自定义类
- 需要在类中实现两个魔法函数，__enter__和__exit__，否则会抛出异常

In [33]:
class FileOpen():
    def __init__(self, filename, mode):
        self.filename = filename
        self.mode = mode

    def __enter__(self):
        self.fp = open(self.filename, self.mode)
        return self.fp

    def __exit__(self, exc_type, exc_val, exc_tb):
        print(exc_type, "|", exc_val, "|", exc_tb)
        self.fp.close()
        # 如果返回True，则会自动吸收异常，不会向外界抛出
        return True

# 注意：如果代码执行成功，没有任何异常，那么exception_type、exception_value以及trackback将会是None，
# 否则的话你可选择处理这个异常或者是直接交给用户处理。如果你想处理这个异常的话，那么必须在__exit__所有结束之后返回True
    
with FileOpen("test.txt", "w") as fp:
    fp.write("hello world")
    a = 1
    b = 0
    c = a / b

<class 'ZeroDivisionError'> | division by zero | <traceback object at 0x0000017043191848>


In [42]:
p = "I love you so much \n I hope you feel so"
d = {}
maxChar = ""
for w in p:
    if w.isspace():
        continue
    if d.get(w) is None:
        d[w] = 1
        if maxChar == "":
            maxChar = w
    else:
        d[w] += 1
        if d[maxChar] < d[w]:
            maxChar = w
            
print(d)
print(maxChar)
print(d[maxChar])

{'I': 2, 'l': 2, 'o': 6, 'v': 1, 'e': 4, 'y': 2, 'u': 3, 's': 2, 'm': 1, 'c': 1, 'h': 2, 'p': 1, 'f': 1}
o
6


## 请简要描述装饰器的作用
- 它本身就是一个函数，主要用于为函数添加额外功能，如插入日志，性能测试

## 如何自定义装饰器

In [50]:
# 定义装饰器

from functools import wraps

# 给装饰器传递参数
def log(flag):
    def decorate(func):
        @wraps(func) #  保证函数签名正确
        # 这里是被装饰的函数传递的参数
        def _wrap(*args, **kwargs):
            try:
                if flag:
                    func(*args, **kwargs)
                    print("name:", func.__name__)
            except Exception as e:
                print(e.args)
        return _wrap
    return decorate

@log(True)
def add(a, b):
    print(f"{a}+{b} = {a+b}")
    
add(2, 3)

执行
2+3 = 5
name: add


## 如何区分调用的是函数还是方法
- 使用type
- 使用isinstance函数，types.MethodType, types.FunctionType

In [57]:
class Person():
    def eat(self):
        pass
    
def eat():
    pass

print(type(Person().eat))
print((type(Person().eat).__name__ == "method"))
print(type(eat))
print((type(eat)).__name__ == "function")


from types import FunctionType, MethodType

print(isinstance(Person().eat,  FunctionType))
print(isinstance(Person().eat,  MethodType))
print(isinstance(eat,  FunctionType))
print(isinstance(eat,  MethodType))

<class 'method'>
True
<class 'function'>
True
False
True
True
False


## 什么是元类，请用代码解释如何使用元类
- 元类，类似于创建类的模板，所有的类都是通过它来创建的，可以自由控制住类的创建过程
    - 比如单例模式，ORM模式


In [13]:
# 单例模式的实现
# 方法一：使用__new__实现,不要使用__instance

class Myclass():
    def __new__(cls, *args, **kwargs):
        if not hasattr(cls, "_instance"):
            cls._instance = super().__new__(cls)
            # 或者如下的写法
#             Myclass._instance = object.__new__(cls)
        return cls._instance
    
    

class Ano(Myclass):
    def __init__(self, name):
        self.name = name
x = Ano("小米")
m = Ano("大米")

print(x.name)
print(m.name)


大米
大米


In [15]:
class Singleton(object):
    def __new__(cls, *args, **kw):
        if not hasattr(cls, '_instance'):
            orig = super(Singleton, cls)
            cls._instance = orig.__new__(cls, *args, **kw)
        return cls._instance

class MyClass(Singleton):
    a = 1


TypeError: object.__new__() takes exactly one argument (the type to instantiate)

In [10]:
# 方法一加锁的写法
import threading
class Singleton(object):
    _instance_lock = threading.Lock()

    def __init__(self, name):
        self.name = name


    def __new__(cls, *args, **kwargs):
        if not hasattr(Singleton, "_instance"):
            with Singleton._instance_lock:
                if not hasattr(Singleton, "_instance"):
                    Singleton._instance = object.__new__(cls)  
        return Singleton._instance

obj1 = Singleton("小明")
obj2 = Singleton("小红")
print(obj1.name)
print(obj2.name)

小红
小红


In [21]:
# 方法二：__new__的另一种写法
class Myclass():
    _instance = None
    
    def __new__(cls, *args, **kwargs):
        if not cls._instance:
            cls._instance = super().__new__(cls)
        return cls._instance
    
    def __init__(self, name):
        self.name = name
    
class HisClass(Myclass):
    pass


# a = Myclass("小强")
# aa = Myclass("小白")
# print(a.name, aa.name)
# 上面的代码依然会出现那种问题

c = Myclass("小丑")
# print(a.name, b.name)
a = HisClass("小强")
b = HisClass("小帅")
print(a.name, b.name)


小白 小白


In [1]:
# 方法三。使用装饰器
from functools import wraps
def Singleton(cls):
    _instance = {}
    @wraps(cls)
    def _singleton(*args, **kwargs):
        if cls not in _instance:
            _instance[cls] = cls(*args, **kwargs)
        return _instance[cls]
    return _singleton

@Singleton
class NewClass():
    def __init__(self, name):
        self.name = name
        
# 使用继承的方法就会和原来的类得到不一样的结果
new_a = NewClass("小丑")
new_b = NewClass("小天")
print(new_a.name, new_b.name)

小丑 小丑


In [20]:
import threading
class Singleton(object):
    _instance_lock = threading.Lock()

    def __init__(self, name):
        self.name = name


    def __new__(cls, *args, **kwargs):
        if not hasattr(Singleton, "_instance"):
            with Singleton._instance_lock:
                if not hasattr(Singleton, "_instance"):
                    Singleton._instance = object.__new__(cls)  
        return Singleton._instance

obj1 = Singleton("xioamnig")
obj2 = Singleton("xiaohong")
print(obj1.name,obj2.name)

xiaohong xiaohong


In [2]:
# 方法四。使用元类实现单例模式
import threading

# 要继承type类
class Singleton_new(type):
    _instance = {}
    _instance_lock = threading.Lock()  #加锁
    
    def __call__(cls, *args, **kwargs):
        if cls not in cls._instance:
            with cls._instance_lock:  # 加锁就多了这句和下面这句以及上面加锁那句
                if cls not in cls._instance:
                    cls._instance[cls] = super(Singleton_new, cls).__call__(*args, **kwargs)
        return cls._instance[cls]
    
class MyClass(metaclass=Singleton_new):
    def __init__(self, name):
        self.name = name
        
xiao = MyClass("小明")
hong = MyClass("小红")
print(xiao.name, hong.name)

小明 小明


# 请用代码说明hasattr，getattr， setattr的作用
- hasattr：可以判断一个对象是否包含某个属性
- getattr：可以获取对象中某一个属性
- setattr：设置对象中某个属性

In [43]:
class Person:
#     name = "人"
    def __init__(self, name, age):
        self.name = name
        self.age =age
    
    def eat(self):
        print("people can eat")
        
person = Person("xiaoming", 23)
print(hasattr(person, "name"))
print(hasattr(Person, "name"))
setattr(person, "name", "小明")
print(getattr(person, "name"))

True
False
小明


# 请阐述 什么是lambda表达式？并用代码描述lambda表达式的应用场景？
- 就是匿名函数 ，一般可以作为函数或方法的参数值使用

In [ ]:
# lambda的使用
a = [('a', 1), ('b', 2), ('c', 3),  ('d', 4)]
a_list = list(map(lambda x: x[0], a))
b_list = list(map(lambda x: x[1], a))
print(a_list)
print(b_list)

# 请描述浅拷贝和深拷贝的区别，请用代码描述？
- 浅拷贝copy：只复制深层对象引用
- 深拷贝deepcopy：复制深层对象本身



In [48]:
import copy

a = [1, 2, 3, ['ab', 'c']]

a1 = copy.copy(a)
a2 = copy.deepcopy(a)
a.append(5)
print(a)
print(a1)
print(a2)
print("=" * 40)

a.remove(5)
a[3].append(3)
print(a)
print(a1)  # 浅拷贝深层对象也会随之改变
print(a2)

[1, 2, 3, ['ab', 'c'], 5]
[1, 2, 3, ['ab', 'c']]
[1, 2, 3, ['ab', 'c']]
[1, 2, 3, ['ab', 'c', 3]]
[1, 2, 3, ['ab', 'c', 3]]
[1, 2, 3, ['ab', 'c']]


# 编写一个生成器，将二维表转换为一维表
- python生成器：
    - yield
- 递归生成器的编写方法与递归函数类似，只是需要处理元素值的时候需要使用yield个关键字

In [55]:
a = [['a', 'b'], ['c', 'd'], ['e', 'f']]

def result(erWeiList):
#     for yiWeiList in erWeiList:
#         yield from yiWeiList

     # 等价于上面的语句
    for yiWeiList in erWeiList:
        for ele in yiWeiList:
            yield ele
       
        
for res in result(a):
    print(res, end=" ")

a b c d e f 

In [61]:
# 使用生成器将任意多维的列转换成一维
a = [2,3,4, [23, 34, [2, 3, [3,5]]]]

def enumList(nestedList):
    try:
        for sublist in nestedList:
            for ele in enumList(sublist):
                yield ele
    except TypeError:
        yield nestedList

res = list(enumList(a))
print(res)

[2, 3, 4, 23, 34, 2, 3, 3, 5]


# 如何获取当前日期的年月日以及是一年中的第几天
- 使用time模块

In [71]:
from datetime import datetime
import time

print(datetime.today())
localtime = time.localtime()
print(localtime)
print(localtime.tm_year)
print(localtime.tm_mon)
print(localtime.tm_mday)
print(localtime.tm_yday)

2021-02-21 22:54:33.937118
time.struct_time(tm_year=2021, tm_mon=2, tm_mday=21, tm_hour=22, tm_min=54, tm_sec=33, tm_wday=6, tm_yday=52, tm_isdst=0)
2021
2
21
52


# python中进程如何通信
-  进程间通信主要包括管道, 系统IPC(包括消息队列,信号量,共享存储), SOCKET

In [73]:
from multiprocessing import Process, Queue
import time, random
import multiprocessing

content = ['java', 'python', 'C语言']


def write(queue):
    for c in content:
        print(f"正在写入==》{c}")
        queue.put_nowait(c)
        time.sleep(random.random())


def read(queue):
    while True:
        if not queue.empty():
            value = queue.get_nowait()
            print(f"正在读取==》{value}")
            time.sleep(random.random())
        else:
            break

# 代码需要在pycharm上运行，且需要使用freeze_support(),否则报错
if __name__ == "__main__":
    multiprocessing.freeze_support()
    queue = Queue()
    write_data = Process(target=write, args=(queue,))
    read_data = Process(target=read, args=(queue,))
    
    write_data.start()
    write_data.join()
    read_data.start()
    read_data.join()
    print("完成")

完成


# 当运行一个线程函数时，如何为该函数传递参数
- 通过Thread类中的args参数传递参数

# 如何创建和使用在线程内部用的全局对象
- threading.local()，local类用于创建一个全局对象，不过该对象只能在线程内部使用，也就是说，全局是针对一个线程而言的

In [ ]:
import threading
import time

# 创建一个全局变量，不过仅针对一个线程而言
a = threading.local()

# 定义一个线程函数
def worker():
    a.x = 0
    for i in range(20):
        time.sleep(0.01)
        a.x += 1
    print("线程正在工作：", threading.currentThread(), "a.x的值是", a.x)
    print()

for i in range(10):
    working = threading.Thread(target=worker)
    working.start()


## 请描述一下什么是协程，请举例说明
- 协程：又称为微线程，纤程，
- 通过async/await语法进行声明，是编写异步应用的推荐方式
- 使用async修饰要运行的函数，在运行协程函数时，需要使用await

## 协程中有哪两个运行任务的函数，如何使用
- run和create_task

In [74]:
# 协程示例(需要在pycharm上运行)
import asyncio

async def main():
    print("hello")
    await asyncio.sleep(1)
    print("world")
    
asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop

In [ ]:
# 使用run运行任务
import asyncio
import time

async def say_after(delay_time, words):
    await asyncio.sleep(delay_time)
    print(words)

async def func():
    print(f"开始时间为：{time.strftime('%X')}")
    await say_after(1, 'hell0')
    await say_after(2, 'world')
    print(f"结束时间为: {time.strftime('%X')}")

asyncio.run(func())

In [ ]:
# 使用create_task运行任务
# 协程示例
import asyncio
import time

async def say_after(delay_time, words):
    await asyncio.sleep(delay_time)
    print(words)

async def func1():
    task1 = asyncio.create_task(say_after(1, 'hello'))
    task2 = asyncio.create_task(say_after(2, 'world'))
    print(f"开始时间为：{time.strftime('%X')}")
    await task1
    await task2
    print(f"结束时间为: {time.strftime('%X')}")

asyncio.run(func1())

# 请解释什么是线程锁，并举例说明如何使用
- 线程锁：目的是将一段代码锁住，，一旦获得锁权限，除非释放线程锁，否则其他任何代码都无法获得锁权限
- 为什么需要线程锁：由于多线程同时在完成特定的操作时，由于并不是原子操作，所以在完成操作的过程中可能会被打断，去做其他的操作。可能会产生脏数据

In [ ]:
from atexit import register
from time import sleep, ctime
import random
from threading import Thread, Lock, currentThread

lock = Lock()

def func():
    lock.acquire()  # 申请锁
    for i in range(5):
        print("Thread Name", "=", currentThread().name, 'i', '=', i)
        sleep(random.randint(1, 2))
    lock.release()

def main():
    for i in range(3):
        Thread(target=func).start()

# register用于在 python 解释器中注册一个退出函数，这个函数在解释器正常终止时自动执行,一般用来做一些资源清理的操作
@register
def exit():
    print("线程运行完毕", ctime())

main()

# 描述一下什么是信号量，如何使用信号量，请举例说明
- 信号量是一个计数器，用于记录资源的消耗情况，当资源消耗时递减，当资源释放时递增，可以认为信号量代表资源是否可用
- 最古老的同步语言之一


In [75]:
# 信号量的使用
from threading import BoundedSemaphore
MAX = 3

seampphore = BoundedSemaphore(MAX)
# 查看剩余的信号量个数
print(seampphore._value)

# 申请资源，即申请信号量,信号量减去1
seampphore.acquire()
print(seampphore._value)
# 若信号量为0，继续申请但又又没释放，那么就会进入阻塞状态
# 可以设置参数blocking=False,那么继续申请就会返回False，申请失败

# 释放信号量，信号量加1
seampphore.release()
print(seampphore._value)
# 若信号量已达到最大的释放值，继续释放就会报错


3
2
3


### 在urls.txt文件中包含了若干个图像url，一行一个url，使用多线程下载这些图像文件，并按url出现的顺序保存为0.jpg, 1.jpb,2.jpg...

In [85]:
from urllib.request import urlretrieve
import threading
import os


with open("图片链接.txt", "r", encoding="utf-8", newline=None) as fp:
    # None表示不输入函数，即相当于 [x for x in your_list if x]，所以空字符会被过滤掉
    # splitlines()会根据换行符进行分割字符串
    pic_url_list = list(filter(None, map(lambda x: x.splitlines()[0], fp.readlines())))


class DownloadThread(threading.Thread):
    def __init__(self, func, args):
        super(DownloadThread, self).__init__(target=func, args=args)

os.chdir("pictures")
def downlaoding(num, url):
    # urlretrieve根据图片url地址进行下载，第二个参数表示保存的文件名
    urlretrieve(url, str(num) + ".jpeg")
    print("图片下载成功")


for i, url in enumerate(pic_url_list):
    try:
        thread = DownloadThread(downlaoding, args=(i, url))
        thread.start()
    except Exception as e:
        print(e.args)


5


# 自定义异常类
- 需要继承Exception


In [1]:
class MyException(Exception):
    pass

num = 0
try:
    if num == 0:
        raise MyException
except MyException:
    print("发生了自定义异常")

发生了自定义异常


## python是否支持多继承，举例说明 
- 支持

## 如果python类的多个父类存在相同的成员，按什么规则处理
- 按MRO列表中的顺序来处理

In [3]:
class Father():
    pass
class Mother():
    pass
class Per(Father, Mother):
    pass

print(Per.__mro__)

(<class '__main__.Per'>, <class '__main__.Father'>, <class '__main__.Mother'>, <class 'object'>)


# 请描述异常捕捉语句中else的作用
- except会在发生异常时执行，而else会在代码正常执行时执行